## Постановка задачи
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Разделите выборку на обучающую/проверочную в соотношении 80/20.

Постройте 2 модели - kNN по 100 соседей и множественную логистическую регрессию - по наиболее оптимальным наборам параметров (для каждой модели), используйте для этого перекрестную проверку GridSearchCV.

Проведите предсказание и проверьте качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)

### Общий набор столбцов для расчета

In [4]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [5]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [6]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [7]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [8]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
58290 -0.607422  0.269287  0.441162 -0.872559  0.611816 -0.169434  0.862305   
9499   0.661133 -0.465576  0.895020  1.226562 -1.634766 -0.169434  0.862305   
20506 -0.278564 -0.220581  1.046875 -0.178467 -1.634766 -0.169434  0.862305   
28610  1.718750  1.249023 -0.240112  1.178711  0.611816 -0.169434 -1.159180   
8283  -0.043610  0.269287 -0.391602 -0.187134  0.611816 -0.169434 -1.159180   

              7         8         9  ...       109       110       111  \
58290  0.043671  0.861328 -0.928711  ... -0.083679  0.441650 -0.149292   
9499  -1.013672  0.864258 -0.928711  ... -0.083679  0.441650 -0.149292   
20506 -1.013672  0.861328 -0.928711  ... -0.083679  0.441650 -0.149292   
28610  1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
8283   1.100586 -1.156250  1.130859  ... -0.083679 -2.263672 -0.149292   

            112       113       114       115       116      117  Response  
582

### Логистическая регрессия
Найдем оптимальный набор столбцов и рассчитаем по нему модель логистической регрессии

In [9]:
def regression_model (columns, df, fit=True):
    x = pd.DataFrame(df, columns=columns)
    model = LogisticRegression(max_iter=1000)
    if fit:
        model.fit(x, y = df["Response"])
    return model

def model_score (columns, df_train, model_func):
    x = pd.DataFrame(df_train, columns=columns)
    model = model_func(columns, df_train, False)
    cv_grid = GridSearchCV(model, {}, cv=5, n_jobs=2,
                scoring=make_scorer(cohen_kappa_score))
    cv_grid.fit(x, df_train["Response"])
    return cv_grid.best_score_

def find_opt_columns (data_train, model_func):
    kappa_score_opt = 0
    columns_opt = []
    for col in columns_transformed:
        kappa_score = model_score([col], data_train, model_func)
        if kappa_score > kappa_score_opt:
            columns_opt = [col]
            kappa_score_opt = kappa_score
    for col in columns_transformed:
        if col not in columns_opt:
            columns_opt.append(col)
            kappa_score = model_score(columns_opt, data_train, model_func)
            if kappa_score < kappa_score_opt:
                columns_opt.pop()
            else:
                kappa_score_opt = kappa_score
    return columns_opt, kappa_score_opt

In [10]:
columns_opt_logr, kappa_score_opt = find_opt_columns(data_train,
                                                    regression_model)
model_logr = regression_model(columns_opt_logr, data_train)
print (kappa_score_opt, columns_opt_logr)

0.3490832174042079 [3, 0, 2, 4, 5, 6, 8, 9, 14, 15, 20, 22, 23, 26, 28, 29, 34, 35, 37, 41, 44, 46, 49, 50, 52, 54, 56, 57, 59, 61, 62, 63, 66, 67, 68, 69, 71, 73, 74, 75, 76, 77, 79, 82, 84, 85, 86, 88, 91, 92, 94, 96, 98, 99, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 116]


### kNN
Посчитаем оптимальную модель для kNN

In [11]:
def knn_model (columns, df_train, fit=True):
    y = data_train["Response"]
    x = pd.DataFrame(df_train, columns=columns)
    model = KNeighborsClassifier(n_neighbors=100)
    if fit:
        model.fit(x, y)
    return model

In [12]:
columns_opt_knn, kappa_score_opt = find_opt_columns(data_train,
                                                    knn_model)
model_knn = knn_model(columns_opt_knn, data_train)
print (kappa_score_opt, columns_opt_knn)

0.3669549281916988 [3, 2, 5, 14, 16, 20, 26, 34, 49, 68, 69, 75, 79, 87, 91, 94, 99, 104]


### Предсказание данных и оценка модели

In [13]:
x_test = pd.DataFrame(data_test, columns=columns_opt_logr)
data_test["target_logr"] = model_logr.predict(x_test)
x_test = pd.DataFrame(data_test, columns=columns_opt_knn)
data_test["target_knn"] = model_knn.predict(x_test)

In [14]:
print ("Логистическая регрессия:",
      round(cohen_kappa_score(data_test["target_logr"],
                data_test["Response"], weights="quadratic"), 3))
print ("kNN, 100:",
      round(cohen_kappa_score(data_test["target_knn"],
                data_test["Response"], weights="quadratic"), 3))

Логистическая регрессия: 0.487
kNN, 100: 0.463


### Матрица неточностей

In [15]:
print ("Логистическая регрессия:\n",
      confusion_matrix(data_test["target_logr"], data_test["Response"]))
print ("kNN:\n",
      confusion_matrix(data_test["target_knn"], data_test["Response"]))

Логистическая регрессия:
 [[ 294  162   12    6   87  142   65   24]
 [ 190  300   10    2  111  133   24   25]
 [   3   13   21    3    7    5    2    0]
 [  15   14   20   82    4   48    6    7]
 [ 104  179   19    2  298  113   48   12]
 [ 247  266  110  148  291  956  287  217]
 [ 106   90    1    2   78  235  421   74]
 [ 309  286   12   37  211  581  722 3578]]
kNN:
 [[ 190  128    7    6   42  108   38    9]
 [ 243  341   12    4  125  132   32   33]
 [   0    0    0    0    0    0    0    0]
 [   3    9    4   32    1   28    0    8]
 [  56  125   34    2  321   92   13    6]
 [ 298  290  133  201  307 1084  283  202]
 [ 163  142    4    5   97  283  602  155]
 [ 315  275   11   32  194  486  607 3524]]
